In [1]:
import ndjson

TRAINING_FILE = "/home/yves/projects/Quill-NLP-Tools-and-Datasets/notw.ndjson"
MAX_SEQ_LENGTH = 100
TRAIN_SIZE = 500000
TEST_SIZE = 5000
BATCH_SIZE = 32


with open(TRAINING_FILE) as i:
    data = ndjson.load(i)
    
data = data[:TRAIN_SIZE]

data = [{"text": item.get("synth_sentence", item.get("orig_sentence")), 
         "entities": item.get("entities", [])} for item in data]


In [2]:
label2idx = {"O": 0}

for sentence in data:
    if "entities" in sentence:
        for (_, _, label) in sentence["entities"]:
            if label not in label2idx:
                label2idx[label] = len(label2idx)
            
print(label2idx)
        

{'O': 0, 'POSSESSIVE': 1, 'VERB': 2, 'ADV': 3, 'WOMAN': 4, 'ITS': 5, 'THEN': 6, 'CHILD': 7}


In [3]:
from typing import List

class BertInputItem(object):
    """ A BertInputItem contains all the information that is needed to finetune
    a Bert model.

    Attributes:
        input_ids: the ids of the input tokens
        input_mask: a list of booleans that indicates what tokens should be masked
        segment_ids: a list of segment ids for the tokens
        label_id: a label id or a list of label ids for the input
    """

    def __init__(self, text: str,
                 input_ids: List[int],
                 input_mask: List[int],
                 segment_ids: List[int],
                 label_ids: List[int]):
        self.text = text
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_ids = label_ids
        

def preprocess_sequence_labelling(examples, label2idx, max_seq_length, tokenizer):
    input_items = []
    for (ex_index, ex) in enumerate(examples):

        # Create a list of token ids
        toks = tokenizer.encode_plus(ex["text"], max_length=max_seq_length, pad_to_max_length=True)
        input_ids = toks["input_ids"]
        segment_ids = toks["token_type_ids"]
        input_mask = toks["attention_mask"]
        
        tokens = tokenizer.convert_ids_to_tokens(input_ids)
        
        if "entities" not in ex:
            labels = [label2idx["O"]] * len(input_ids)
        else:
            labels = [label2idx["O"]]
            cur_index = 0
            for num, tok in enumerate(tokens[1:]):

                if num > 0 and not tok.startswith("##"):
                    cur_index += 1

                found_entity = False
                for entity in ex["entities"]:
                    if cur_index >= entity[0] and cur_index <= entity[1]:
                        labels.append(label2idx[entity[2]])
                        found_entity = True
                if not found_entity:
                    labels.append(label2idx["O"])


                if tok.startswith("##"):
                    cur_index += len(tok)-2
                else:
                    cur_index += len(tok)
        
        assert len(labels) == len(input_ids)
        
        input_items.append(
            BertInputItem(text=ex["text"],
                          input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_ids=labels))
    return input_items

In [4]:
from transformers import BertForTokenClassification
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
input_items = preprocess_sequence_labelling(data, label2idx, MAX_SEQ_LENGTH, tokenizer)



I0317 15:32:54.715602 140152479643456 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0317 15:32:55.623711 140152479643456 file_utils.py:57] TensorFlow version 2.1.0 available.
I0317 15:32:56.305696 140152479643456 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at /home/yves/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [5]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

def get_data_loader(input_items: List[BertInputItem], batch_size: int, shuffle: bool=True) -> DataLoader:
    """
    Constructs a DataLoader for a list of BERT input items.

    Args:
        input_items: a list of BERT input items
        batch_size: the batch size
        shuffle: indicates whether the data should be shuffled or not.

    Returns: a DataLoader for the input items

    """
    all_input_ids = torch.tensor([f.input_ids for f in input_items], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in input_items], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in input_items], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_ids for f in input_items], dtype=torch.long)
    
    data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

    dataloader = DataLoader(data, shuffle=shuffle, batch_size=batch_size)

    return dataloader

In [6]:
import random

random.shuffle(input_items)

test_items = input_items[-TEST_SIZE:]
valid_items = input_items[-2*TEST_SIZE:-TEST_SIZE]
train_items = input_items[:-2*TEST_SIZE]

test_dl = get_data_loader(test_items, BATCH_SIZE, shuffle=False)
dev_dl = get_data_loader(valid_items, BATCH_SIZE, shuffle=False)
train_dl = get_data_loader(train_items, BATCH_SIZE, shuffle=True)


In [7]:
import sys
sys.path.append('../')

from quillnlp.models.bert.train import train
from transformers import BertModel

model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label2idx))
model.to("cuda")

train(model, train_dl, dev_dl, BATCH_SIZE, 32/BATCH_SIZE, device="cuda", num_train_epochs=10)

I0317 15:33:27.167923 140152479643456 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/yves/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
I0317 15:33:27.169666 140152479643456 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddin


Loss history: []
Dev loss: 0.5565451331138611


Epoch:  10%|█         | 1/10 [22:41<3:24:16, 1361.85s/it]


Loss history: [0.5565451331138611]
Dev loss: 0.21802658158540725


Epoch:  20%|██        | 2/10 [45:25<3:01:39, 1362.41s/it]

...

Epoch:  90%|█████████ | 9/10 [3:24:06<22:40, 1360.79s/it]


Loss history: [0.5565451331138611, 0.21802658158540725, 0.15455146169662476, 0.127263275295496, 0.09640713724493981, 0.09293479809165, 0.09161961454153061, 0.0920342872440815]
Dev loss: 0.0960927545428276


Epoch: 100%|██████████| 10/10 [3:46:48<00:00, 1361.17s/it]


Loss history: [0.5565451331138611, 0.21802658158540725, 0.15455146169662476, 0.127263275295496, 0.09640713724493981, 0.09293479809165, 0.09161961454153061, 0.0920342872440815, 0.0960927545428276]
Dev loss: 0.0944093667715788


'/tmp/model.bin'

In [8]:
from quillnlp.models.bert.train import evaluate

output_model_file = "/tmp/model.bin"
print("Loading model from", output_model_file)
device="cpu"

model_state_dict = torch.load(output_model_file, map_location=lambda storage, loc: storage)
model = BertForTokenClassification.from_pretrained("bert-base-cased", state_dict=model_state_dict, num_labels=len(label2idx))
model.to(device)

#_, train_correct, train_predicted = evaluate(model, train_dataloader)
#_, dev_correct, dev_predicted = evaluate(model, dev_dataloader)
_, _, test_correct, test_predicted = evaluate(model, test_dl, device)

Loading model from /tmp/model.bin


I0317 19:20:22.996674 140152479643456 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at /home/yves/.cache/torch/transformers/b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.3d5adf10d3445c36ce131f4c6416aa62e9b58e1af56b97664773f4858a46286e
I0317 19:20:22.998143 140152479643456 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddin

In [9]:
idx2label = {v:k for k,v in label2idx.items()}

for item, correct, predicted in zip(test_items, test_correct, test_predicted):
    print(item.text)
    for error in set(predicted):
        print("Found:", idx2label[error])
    for error in set(correct):
        print("Correct:", idx2label[error])
    

He explains why he thinks the version of the " The Star-Spangled Banner " being played in Rio sounds more melancholy than usual.
Found: O
Correct: O
It wasn't anti-Manafort.
Found: O
Correct: O
If paused, you'll be notified of the number of additional comments that have come in.
Found: O
Correct: O
But it was not to last.
Found: O
Found: VERB
Correct: O
She's been called a weapon in the wardrobes of women in power.
Found: O
Correct: O
If paused, you'll be notified of the number of additional comment's that have come in.
Found: O
Found: POSSESSIVE
Correct: O
Correct: POSSESSIVE
Hey doctor love thanks you so so much for making the time that it NASA astronaut doctor Sam lovely.
Found: O
Found: VERB
Correct: O
Correct: VERB
At the end of the summer, three randomly selected families that have sent in photos will receive a prize package of books and KidsPost goodies.
Found: O
Correct: O
We woulds like to take a moment to remind you that other folks actual appeared onstage at the 2016 MTV Vid

Correct: VERB
" First the water.
Found: O
Correct: O
" I learnt from him and
Found: O
Correct: O
Has a stable income.
Found: O
Correct: O
Correct: VERB
" Four people are under the rubble, but they are not showing any sign of life.
Found: O
Correct: O
Could this am more obvious?
Found: O
Found: VERB
Correct: O
Correct: VERB
But Obama has done good in his two elections among Cuban Americans, signaling that younger Cubans may be more open to easing travel rule's.
Found: O
Found: POSSESSIVE
Found: ADV
Correct: O
Correct: POSSESSIVE
Correct: ADV
I musts have more energy.
Found: O
Found: VERB
Correct: O
Correct: VERB
Rio 2016 officials said army and military police's will be increased in the area after the incident.
Found: O
Found: POSSESSIVE
Correct: O
Correct: POSSESSIVE
The so-called Collar City is now frayed, but back in the mid-19th century it was a prosperous transportation nexus across the Hudson River from Albany, with a steel industry and shirt factories and scientific research faci

In [10]:
# Read error input

error_map = {'POSSESSIVE': 'Plural versus possessive nouns', 
             'VERB': 'Subject-verb agreement', 
             'ADV': 'Adverbs versus adjectives', 
             'ITS': "Its versus it's", 
             'THEN': 'Than versus then'}

data = []
with open("../tests/data/grammar_valid.tsv") as i:
    for line in i:
        line = line.strip().split("\t")
        if len(line) == 3:
            data.append(line)

results = {}
for (error_type, correct_sentence, incorrect_sentence) in data:
    if error_type not in results:
        results[error_type] = {"cc": 0, "ci": 0, "ic": 0, "ii": 0}

    correct_features = preprocess_sequence_labelling([{"text": correct_sentence}], 
                                                     label2idx, MAX_SEQ_LENGTH, tokenizer)

    incorrect_features = preprocess_sequence_labelling([{"text": incorrect_sentence}],
                                                       label2idx, MAX_SEQ_LENGTH, tokenizer)

    correct_dl = get_data_loader(correct_features, 1, shuffle=False)
    incorrect_dl = get_data_loader(incorrect_features, 1, shuffle=False)
    
    _, _, _, correct_predicted = evaluate(model, correct_dl, device)
    _, _, _, incorrect_predicted = evaluate(model, incorrect_dl, device)
        
    errors = [idx2label[i] for i in set(correct_predicted[0])]
    errors = [error_map[e] for e in errors if e in error_map]
    print(correct_sentence)
    print(errors)

    if len(errors) == 0:
        results[error_type]["cc"] += 1
    else:
        results[error_type]["ci"] += 1

    errors = [idx2label[i] for i in set(incorrect_predicted[0])]
    errors = [error_map[e] for e in errors if e in error_map]
    print(incorrect_sentence)
    print(errors)

    if len(errors) == 0:
        results[error_type]["ic"] += 1
    else:
        results[error_type]["ii"] += 1


He sorted through a stack of documents, took one out, glanced at it, then set it aside.
[]
He sorted through a stack of documents, took one out, glanced at it, than set it aside.
['Than versus then']



If he had ridden a female donkey, then that would have been a sign that he was following in the footsteps of Father Abraham.
[]
If he had ridden a female donkey, than that would have been a sign that he was following in the footsteps of Father Abraham.
['Than versus then']



But then the steward asked if I wanted breakfast.
[]
But than the steward asked if I wanted breakfast.
[]



The more resources we have by then, the deeper we can dig.
[]
The more resources we have by than, the deeper we can dig.
['Than versus then']



It lay in my hand a second, warm and still, and then sudden as that leapt away.
['Adverbs versus adjectives']
It lay in my hand a second, warm and still, and than sudden as that leapt away.
['Than versus then']


...


This fight became known as the civil rights movement.
[]
This fight became known as the civil right's movement.
['Plural versus possessive nouns']



Some states have standards as well.
[]
Some state's have standards as well.
['Plural versus possessive nouns']



Most American government textbooks contain citations of numerous cases.
[]
Most American government textbooks contain citations of numerous case's.
['Plural versus possessive nouns']



We can't stop our lives from changing, but we can cherish past experiences.
[]
We can't stop our live's from changing, but we can cherish past experiences.
['Plural versus possessive nouns']



Nevertheless, the faint glow on two areas of the trunk mat prompted them to mark those spots for further testing.
[]
Nevertheless, the faint glow on two area's of the trunk mat prompted them to mark those spots for further testing.
['Plural versus possessive nouns']


In [11]:
results

{'Than versus then': {'cc': 18, 'ci': 2, 'ic': 5, 'ii': 15},
 'Subject-verb agreement': {'cc': 110, 'ci': 10, 'ic': 22, 'ii': 98},
 "Its versus it's": {'cc': 35, 'ci': 5, 'ic': 20, 'ii': 20},
 'Adverbs versus adjectives': {'cc': 20, 'ci': 0, 'ic': 10, 'ii': 10},
 'Plural versus possessive nouns': {'cc': 19, 'ci': 1, 'ic': 0, 'ii': 20}}

In [12]:
for error_type in results:
    correct = results[error_type]["cc"] + results[error_type]["ii"]
    incorrect = results[error_type]["ic"] + results[error_type]["ci"]

    precision = results[error_type]["cc"]/ (results[error_type]["cc"] + results[error_type]["ic"])
    recall = results[error_type]["cc"] / (results[error_type]["cc"] + results[error_type]["ci"])
    fscore = 2*precision*recall/(precision+recall)

    print(error_type, precision, recall, fscore)

Than versus then 0.782608695652174 0.9 0.8372093023255814
Subject-verb agreement 0.8333333333333334 0.9166666666666666 0.8730158730158729
Its versus it's 0.6363636363636364 0.875 0.7368421052631579
Adverbs versus adjectives 0.6666666666666666 1.0 0.8
Plural versus possessive nouns 1.0 0.95 0.9743589743589743
